### Creating and Persisting an ML Model

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [2]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [3]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

Create a subset of features as an example.

In [4]:
include = ['failures', 'schoolsup', 'internet', 'studytime', 'absences','Medu','Fedu','paid','famsup','G3']
df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [5]:
df.info

<bound method DataFrame.info of      Medu  Fedu  studytime  failures schoolsup famsup paid internet  absences  \
0       4     4          2         0       yes     no   no       no         6   
1       1     1          2         0        no    yes   no      yes         4   
2       1     1          2         3       yes     no  yes      yes        10   
3       4     2          3         0        no    yes  yes      yes         2   
4       3     3          2         0        no    yes  yes       no         4   
..    ...   ...        ...       ...       ...    ...  ...      ...       ...   
390     2     2          2         2        no    yes  yes       no        11   
391     3     1          1         0        no     no   no      yes         3   
392     1     1          1         3        no     no   no       no         3   
393     3     2          1         0        no     no   no      yes         0   
394     1     1          1         0        no     no   no      yes         5

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [6]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [7]:
df['schoolsup'] = np.where(df['schoolsup']=='yes', 1, 0)

In [10]:
df['paid'] = np.where(df['paid']=='yes', 1, 0)

In [8]:
df['internet'] = np.where(df['internet']=='yes', 1, 0)

In [11]:
df['famsup'] = np.where(df['famsup']=='yes', 1, 0)

In [12]:
df.describe()

,Medu,Fedu,studytime,failures,schoolsup,famsup,paid,internet,absences,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,2.749367,2.521519,2.035443,0.334177,0.129114,0.612658,0.458228,0.832911,5.708861,10.415190,0.184810
std,1.094735,1.088201,0.839240,0.743651,0.335751,0.487761,0.498884,0.373528,8.003096,4.581443,0.388636
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,8.000000,0.000000
50%,3.000000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,4.000000,11.000000,0.000000
75%,4.000000,3.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,8.000000,14.000000,0.000000
max,4.000000,4.000000,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,75.000000,20.000000,1.000000


Drop the G3 score

In [13]:
include = ['failures', 'schoolsup', 'internet', 'studytime', 'absences','Medu','Fedu','paid','famsup','qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

Import scikit-learn and build a random forest classifer

In [14]:
from sklearn.ensemble import RandomForestClassifier as rf
import sklearn
dependent_variable = 'qual_student'
x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]
clf = rf(n_estimators = 1000)
clf.fit(x, y)

RandomForestClassifier(n_estimators=1000)

In [15]:
pred = clf.predict(x)
sklearn.metrics.f1_score(y, pred, average='binary')

0.920863309352518

It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [16]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, 'app/handlers/model.pkl')

['app/handlers/model.pkl']

In [95]:
query_df = pd.DataFrame({ 'failures' : pd.Series(1) ,'schoolsup' : 1 ,'internet' : 1, 'studytime' : pd.Series(3), 'absences' : pd.Series(14)})

In [96]:
pred = clf.predict(query_df)

/home/leftovercode/.local/share/virtualenvs/fall-22-hw4-team-crepe-6Pw8PdTl/lib/python3.8/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- G3

  warnings.warn(message, FutureWarning)


ValueError: X has 5 features, but RandomForestClassifier is expecting 6 features as input.

In [ ]:
x

In [ ]:
type(x)